## PUNTO # 3

[35p] Recuperación ranqueada y vectorización de documentos (RRDV)

Importación de librerías requeridas

In [1]:
import nltk
import numpy as np
import pandas as pd

Definición de paths requeridos para la correcta ejecución del ejercicio

In [4]:
##################
## Paths para apuntar a la data
###################
path_docs = './data/docs-raw-texts' #path de los documentos
path_queries = './data/queries-raw-texts'
ground_truth_path = 'C:/Users/DELL/Documents/relevance-judgments.tsv'  #path de lectura del ground-truth
salidaFile = "RRDV-consultas_resultados.tsv"  #path para exportación de resultados

# Fase de preprocesamiento del corpus.

Este código preprocesa documentos y consultas mediante:

Tokenización: Divide el texto en palabras.

Eliminación de palabras vacías: Filtra palabras comunes irrelevantes (stopwords).

Lematización: Reduce las palabras a su forma base y cuenta su frecuencia.

Extracción de términos: Genera un conjunto de términos únicos en todos los documentos.

Al final, aplica estos pasos a los documentos en un directorio y cuenta el número total de documentos procesados.

In [5]:
#  Pasos de preprocesamiento: para los siguientes puntos,
#  debe preprocesar documentos y consultas mediante tokenización a nivel de palabra,
#  eliminación de palabras vacías, normalización y stemming

import os
from pathlib import Path
import re

class procesamientotexto:

    def __init__(self,path_dir):
        self.path = path_dir
        self.tokens_doc = {} 
        self.word_tok_nltk_es_sw = {}
        self.nltk_lemmaList = {}

    def tokenizacion(self):
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        for doc in os.listdir(self.path):
            path = os.path.join(self.path,doc)
            content_archivo = open(path,encoding='utf8').read()
            texto = re.match('[\w\W]+<raw><!\[CDATA\[(?P<texto>(.|\n|\s|\s)+)\]\]></raw>',content_archivo).groupdict()['texto'].lower()
            # self.tokens_doc[doc] = nltk.word_tokenize(texto,preserve_line=True)
            self.tokens_doc[doc] = tokenizer.tokenize(texto)
            
        return self.tokens_doc
    
    def stopwords(self):
        nltk_stop_words_es = set(nltk.corpus.stopwords.words('english'))
        for name_doc,doc in self.tokens_doc.items():
            self.word_tok_nltk_es_sw[name_doc] = [token for token in doc if token not in nltk_stop_words_es ]
        return self.word_tok_nltk_es_sw
    
    def stemming(self): 
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        index = 0
        for name_doc, doc in self.word_tok_nltk_es_sw.items():
            index += 1
            term_count = {}
            lemmatized_text = []
            for word in doc:
                lemmatized_word = wordnet_lemmatizer.lemmatize(word)
                lemmatized_text.append(lemmatized_word)
                if lemmatized_word in term_count:
                    term_count[lemmatized_word] += 1
                else:
                    term_count[lemmatized_word] = 1
            self.nltk_lemmaList[name_doc] = {
                'index': int(re.match('wes2015.(d|q)(?P<num>\d+).naf', name_doc).groupdict()['num']),
                'text': lemmatized_text,
                'term_count': term_count
            }
        return self.nltk_lemmaList
    
    def dicterminos(self):
        dic = set([])
        for doc in self.nltk_lemmaList.values():
            dic = dic.union(set(doc['text'])) 
        return dic

    
text_process = procesamientotexto(path_docs)
doc_tokens = text_process.tokenizacion()
word_tok_nltk_es_sw = text_process.stopwords()
nltk_lemmaList = text_process.stemming()  # los textos lemmatizados
dicterminos = text_process.dicterminos()  # vocabulario

# Número total de documentos
N = len(nltk_lemmaList)

print(nltk_lemmaList['wes2015.d001.naf'])
print(dicterminos)
print("Documentos: ", N)


<>:22: SyntaxWarning: invalid escape sequence '\w'
<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\w'
<>:49: SyntaxWarning: invalid escape sequence '\d'
C:\Users\juane\AppData\Local\Temp\ipykernel_33332\2508944858.py:22: SyntaxWarning: invalid escape sequence '\w'
  texto = re.match('[\w\W]+<raw><!\[CDATA\[(?P<texto>(.|\n|\s|\s)+)\]\]></raw>',content_archivo).groupdict()['texto'].lower()
C:\Users\juane\AppData\Local\Temp\ipykernel_33332\2508944858.py:49: SyntaxWarning: invalid escape sequence '\d'
  'index': int(re.match('wes2015.(d|q)(?P<num>\d+).naf', name_doc).groupdict()['num']),


{'index': 1, 'text': ['william', 'beaumont', 'human', 'digestion', 'william', 'beaumont', 'physiology', 'digestion', 'image', 'source', 'november', '21', '1785', 'u', 'american', 'surgeon', 'william', 'beaumont', 'born', 'became', 'best', 'known', 'father', 'gastric', 'physiology', 'following', 'research', 'human', 'digestion', 'william', 'beaumont', 'born', 'lebanon', 'connecticut', 'became', 'physician', 'served', 'surgeon', 'mate', 'army', 'war', '1812', 'opened', 'private', 'practice', 'plattsburgh', 'new', 'york', 'rejoined', 'army', 'surgeon', '1819', 'beaumont', 'stationed', 'fort', 'mackinac', 'mackinac', 'island', 'michigan', 'early', '1820s', 'existed', 'protect', 'interest', 'american', 'fur', 'company', 'fort', 'became', 'refuge', 'wounded', '19', 'year', 'old', 'french', 'canadian', 'fur', 'trader', 'named', 'alexis', 'st', 'martin', 'shotgun', 'went', 'accident', 'american', 'fur', 'company', 'store', 'close', 'range', 'june', '6th', '1822', 'st', 'martin', 'wound', 'quit

# Implementación del indice invertido
Este código construye un índice invertido que asocia cada término con los documentos en los que aparece y cuenta cuántas veces aparece en total.

In [6]:
# Implementación del índice invertido usando 
# los 331 documentos en el conjunto de datos.

def indiceinvertido(doc_lemalist: dict,terminos:dict):
    indiceinvertido = {}
    for termino in terminos: 
         indiceinvertido[termino] = {'IDdocs':[],'len':0}
  
    for documento in doc_lemalist.values():  
        set_texto = set(documento['text'])
        for termino in set_texto: 
            indiceinvertido[termino]['IDdocs'].append(documento['index'])
            indiceinvertido[termino]['len'] +=1 
    return indiceinvertido


list_indiceinvertido = indiceinvertido(nltk_lemmaList,dicterminos)
print(list_indiceinvertido)

{'mix': {'IDdocs': [85, 228], 'len': 2}, 'child': {'IDdocs': [1, 2, 9, 10, 14, 16, 17, 24, 38, 42, 44, 53, 55, 58, 64, 67, 71, 73, 76, 81, 84, 85, 88, 89, 90, 93, 100, 102, 105, 109, 111, 115, 118, 121, 133, 135, 136, 137, 143, 150, 158, 161, 164, 167, 175, 179, 183, 184, 189, 190, 191, 193, 194, 198, 200, 205, 206, 212, 215, 227, 229, 230, 241, 249, 252, 254, 262, 279, 283, 293, 294, 305, 309, 310, 314, 316, 324], 'len': 77}, 'selling': {'IDdocs': [51, 54, 65, 84, 188, 233], 'len': 6}, 'deaf': {'IDdocs': [231, 289], 'len': 2}, 'azores': {'IDdocs': [176], 'len': 1}, 'culpeper': {'IDdocs': [35], 'len': 1}, 'organising': {'IDdocs': [100, 191], 'len': 2}, 'au': {'IDdocs': [325], 'len': 1}, 'vigorous': {'IDdocs': [162], 'len': 1}, 'quarter': {'IDdocs': [18, 55, 99, 177, 188, 299], 'len': 6}, 'sandymount': {'IDdocs': [136], 'len': 1}, 'manhattan': {'IDdocs': [5, 28, 64, 81, 103, 108, 110, 121, 205, 251, 254, 271, 280], 'len': 13}, 'blackmailed': {'IDdocs': [77], 'len': 1}, 'solitary': {'IDd

# Creación de una representación vectorial a partir de la generación del indice invertido, como fase previa de la fase de procesamiento de similitud coseno
Este código calcula la representación TF-IDF de documentos. Recorre términos y documentos, computando TF-IDF para cada uno. Es preciso pero puede ser costoso en tiempo para grandes colecciones.

In [7]:
# [10p] Cree una función que, a partir del índice invertido, 
# cree la representación vectorial ponderada tf.idf de un documento o consulta. 
# Describa en detalle su estrategia, ¿es eficiente? ¿por qué si, por qué no?

import numpy as np

def calcular_tf_idf(nltk_lemmaList, dicterminos, list_indiceinvertido, N):
    tf_idf = {}

    for doc_name, doc_data in nltk_lemmaList.items():
        tf_idf[doc_name] = {}
        
        for term in dicterminos:
            tf = doc_data['term_count'].get(term, 0)
            df = list_indiceinvertido[term]['len']

            if tf > 0 and df > 0:
                # Cálculo de TF y DF según las fórmulas proporcionadas
                tf_value = np.log10(1 + tf)
                df_value = np.log10(N / df)
                
                # Cálculo de TF-IDF
                tf_idf[doc_name][term] = tf_value * df_value
            else:
                tf_idf[doc_name][term] = 0.0

    return tf_idf

# Calcular la representación TF-IDF
tf_idf_representation = calcular_tf_idf(nltk_lemmaList, dicterminos, list_indiceinvertido, N)
print(tf_idf_representation['wes2015.d001.naf'])
print(tf_idf_representation['wes2015.d002.naf'])

{'mix': 0.0, 'child': 0.19065351522147012, 'selling': 0.0, 'deaf': 0.0, 'azores': 0.0, 'culpeper': 0.0, 'organising': 0.0, 'au': 0.0, 'vigorous': 0.0, 'quarter': 0.0, 'sandymount': 0.0, 'manhattan': 0.0, 'blackmailed': 0.0, 'solitary': 0.0, 'hub': 0.0, 'marc': 0.0, 'aunou': 0.0, 'connecting': 0.0, 'iudiciis': 0.0, 'cocteau': 0.0, 'axiomatic': 0.0, 'wink': 0.0, 'différentiel': 0.0, 'industrialist': 0.0, 'emergency': 0.0, 'geneticist': 0.0, 'loewe': 0.0, 'dispossessed': 0.0, 'resource': 0.0, 'humble': 0.0, 'omnimodus': 0.0, 'increase': 0.0, 'influenced': 0.0, 'maneuvering': 0.0, 'chemotherapy': 0.0, 'siècle': 0.0, 'superior': 0.0, 'economics': 0.0, 'guide': 0.0, 'uc': 0.0, 'optical': 0.0, 'salad': 0.0, 'crochet': 0.0, 'practicable': 0.0, 'bellevue': 0.0, 'monitoring': 0.0, 'purely': 0.0, 'p': 0.0, 'contagion': 0.0, 'heated': 0.0, 'businesswoman': 0.0, 'vincent': 0.0, 'container': 0.0, 'kenton': 0.0, 'terry': 0.0, 'bought': 0.0, 'switched': 0.0, 'jános': 0.0, 'cattle': 0.0, 'physic': 0.0,

Con la representación vectorial generada, esta sección lo representa a través de un DataFrame para facilidad de lectura.


In [8]:
# Función para crear un dataframe que permita visualizar los datos para cada termino, 
# y su ponderación TF-IDF en un dataFrame para cada documento

def generar_vector_tf_idf_df(tf_idf_representation, doc_name, dicterminos):
    data = [{'Término': term, 'TF-IDF': tf_idf_representation[doc_name].get(term, 0.0)} for term in dicterminos]
    df = pd.DataFrame(data)
    return df

# Ejemplo de uso para el documento 'wes2015.d001.naf'
doc_name = 'wes2015.d001.naf'
df_vector_tf_idf = generar_vector_tf_idf_df(tf_idf_representation, doc_name, dicterminos)
print(df_vector_tf_idf)

            Término    TF-IDF
0               mix  0.000000
1             child  0.190654
2           selling  0.000000
3              deaf  0.000000
4            azores  0.000000
...             ...       ...
16834    wróblewski  0.000000
16835         oskar  0.000000
16836   manufacture  0.000000
16837       moronic  0.000000
16838  monkeyshines  0.000000

[16839 rows x 2 columns]


Esta sección genera un vector de documento del TF-ID

In [9]:
# Función de apoyo para crear vector de documento TF-IDF

def generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos):
    vector_tf_idf = []   
    for term in dicterminos:
        tf_idf_value = tf_idf_representation[doc_name].get(term, 0.0)
        vector_tf_idf.append(tf_idf_value) 
    return vector_tf_idf

# Ejemplo de uso para el documento 'wes2015.d001.naf'
doc_name = 'wes2015.d001.naf'
vector_tf_idf = generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos)
print(vector_tf_idf)

[0.0, 0.19065351522147012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22769908878428896, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16881358820430456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5041439507191523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35735236035046253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

# Procesamiento de la similitud coseno entre dos vectores con la representación TF-IDF de dos documentos
Este código calcula la similitud coseno entre dos documentos usando sus vectores TF-IDF. Devuelve un valor entre -1 y 1, indicando cuán similares son.

In [10]:
# [10p] Cree una función que reciba dos vectores de documentos y calcule la similitud del coseno.

from numpy.linalg import norm

def simil_Coseno(doc1, doc2, tf_idf_representation, dicterminos):
    
    vector1 = generar_vector_tf_idf(tf_idf_representation, doc1, dicterminos)
    vector2 = generar_vector_tf_idf(tf_idf_representation, doc2, dicterminos)
    arr1 = np.array(vector1)
    arr2 = np.array(vector2)
    
    if len(arr1)==len(arr2):
        coseno = np.dot(arr1,arr2)/(norm(arr1)*norm(arr2))
    else:
        coseno = "error de comparación"   
    return coseno
    
# Ejemplo de uso
DOC1_name = 'wes2015.d001.naf'
DOC2_name = 'wes2015.d002.naf'
CosenoSimil = simil_Coseno(DOC1_name, DOC2_name, tf_idf_representation, dicterminos)
print("Similitud Coseno:\n", CosenoSimil)

Similitud Coseno:
 0.012657298971615048


# Recuperación de los documentos clasificados
Esta sección, procede a tomar las queries mediante el archivo externo, realizando a este conjunto de datos un preprocesamiento de texto, y posterioremente procesar para generar una representación vectorial del TF-IDF de cada documento.


In [11]:
# [5p] Para cada una de las 35 consultas en el conjunto de datos, 
# recupere los documentos clasificados -ordenados por el puntaje de similitud del coseno-
# (incluya solo los documentos con un puntaje superior a 0 para una consulta determinada). 
# Escriba un archivo (RRDV-consultas_resultados) con los resultados siguiendo el siguiente formato:
# q01 dXX: cos_simi(q01,dXX),dYY: cos_simi(q01, dYY),dZZ: cos_simi(q01,dZZ)…

proc_querys = procesamientotexto(path_queries)
tokens_querys = proc_querys.tokenizacion()
tokens_querys_sw = proc_querys.stopwords()
terminos_querys = proc_querys.stemming()
print(terminos_querys)
Nqueries = len(terminos_querys)
print(Nqueries)

# Calcular la representación TF-IDF para los queries
doc_query = 'wes2015.q01.naf'
tf_idf_rep_queries = calcular_tf_idf(terminos_querys, dicterminos, list_indiceinvertido, N)
# Ejemplo de uso para el documento 'wes2015.q01.naf'
df_vectorQuery_tf_idf = generar_vector_tf_idf_df(tf_idf_rep_queries, doc_query, dicterminos)
df_vectorQuery_tf_idf


{'wes2015.q01.naf': {'index': 1, 'text': ['fabrication', 'music', 'instrument'], 'term_count': {'fabrication': 1, 'music': 1, 'instrument': 1}}, 'wes2015.q02.naf': {'index': 2, 'text': ['famous', 'german', 'poetry'], 'term_count': {'famous': 1, 'german': 1, 'poetry': 1}}, 'wes2015.q03.naf': {'index': 3, 'text': ['romanticism'], 'term_count': {'romanticism': 1}}, 'wes2015.q04.naf': {'index': 4, 'text': ['university', 'edinburgh', 'research'], 'term_count': {'university': 1, 'edinburgh': 1, 'research': 1}}, 'wes2015.q06.naf': {'index': 6, 'text': ['bridge', 'construction'], 'term_count': {'bridge': 1, 'construction': 1}}, 'wes2015.q07.naf': {'index': 7, 'text': ['walk', 'fame', 'star'], 'term_count': {'walk': 1, 'fame': 1, 'star': 1}}, 'wes2015.q08.naf': {'index': 8, 'text': ['scientist', 'worked', 'atomic', 'bomb'], 'term_count': {'scientist': 1, 'worked': 1, 'atomic': 1, 'bomb': 1}}, 'wes2015.q09.naf': {'index': 9, 'text': ['invention', 'internet'], 'term_count': {'invention': 1, 'inte

,Término,TF-IDF
0,mix,0.0
1,child,0.0
2,selling,0.0
3,deaf,0.0
4,azores,0.0
...,...,...
16834,wróblewski,0.0
16835,oskar,0.0
16836,manufacture,0.0
16837,moronic,0.0


# Calculo de la similitud coseno entre consultas y documentos, ordena los resultados, y los guarda en un archivo y en un DataFrame.
En esta sección, ya se procede a realizar el procesamiento de información entre la representación vectorial del corpus, contra la representacióon vectorial de los queries, mediante la tecnica de similitud coseno, al final de esta sección se generará una representación que se exporta a un archivo externo, y también cuenta con una representación en memoria mediante un DataFrame para proximas operaciones

In [12]:
# Desarrollo para evaluar el conjunto de queries con el conjunto de documentos

def simil_Coseno(vector1, vector2):
    arr1 = np.array(vector1)
    arr2 = np.array(vector2)
    
    if len(arr1) == len(arr2):
        coseno = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))
    else:
        coseno = 0 
    return coseno

def generar_similitudes_y_dataframe(tf_idf_representation, tf_idf_rep_queries, dicterminos):
    resultados = []
    data = []  
    for q_name, q_vector in tf_idf_rep_queries.items():
        q_id = f"q{int(q_name.split('.')[1][1:]):02d}" 
        similitudes = []
        for doc_name, doc_vector in tf_idf_representation.items():
            doc_id = f"d{int(doc_name.split('.')[1][1:]):02d}"  
            coseno_sim = simil_Coseno(generar_vector_tf_idf(tf_idf_representation, doc_name, dicterminos),
                                      generar_vector_tf_idf(tf_idf_rep_queries, q_name, dicterminos))
            if coseno_sim > 0:
                similitudes.append((doc_id, coseno_sim, q_id))
        similitudes.sort(key=lambda x: x[1], reverse=True)
        similitudes_str = ",".join([f"{doc_id}:{similitudes:.4f}({q_id},{doc_id})" 
                                     for doc_id, similitudes, q_id in similitudes])
        if similitudes_str:
            resultados.append(f"{q_id}\t{similitudes_str}")  
            data.append({'query_id': q_id, 'docs': similitudes_str})
    resultados_df = pd.DataFrame(data)
   
    return resultados, resultados_df

def escribir_resultados(resultados, salida):
    with open(salida, 'w', encoding='utf-8') as file:
        for resultado in resultados:
            file.write(resultado + "\n\n")  

resultados, resultados_df = generar_similitudes_y_dataframe(tf_idf_representation, tf_idf_rep_queries, dicterminos)
escribir_resultados(resultados, salidaFile)
resultados_df

,query_id,docs
0,q01,"d170:0.0845(q01,d170),d85:0.0752(q01,d85),d254..."
1,q02,"d147:0.1211(q02,d147),d149:0.0898(q02,d149),d2..."
2,q03,"d283:0.0727(q03,d283),d291:0.0663(q03,d291),d1..."
3,q04,"d270:0.1238(q04,d270),d19:0.1187(q04,d19),d310..."
4,q06,"d329:0.2327(q06,d329),d297:0.2205(q06,d297),d2..."
5,q07,"d146:0.1223(q07,d146),d04:0.1066(q07,d04),d289..."
6,q08,"d110:0.1514(q08,d110),d251:0.1328(q08,d251),d1..."
7,q09,"d198:0.1531(q09,d198),d199:0.1267(q09,d199),d2..."
8,q10,"d231:0.0691(q10,d231),d60:0.0652(q10,d60),d100..."
9,q12,"d277:0.1723(q12,d277),d258:0.1296(q12,d258),d1..."


# Tratamiento del "Ground-Truth"
En esta parte del desarrollo propuesto, se procede a importar los datos del "ground-truth", con el fin de evaluar y validar los resultados hasta ahora procesados mediante las metricas de evaluación propuestas.

In [85]:
# [10p] Evaluación de resultados. Calcule P@M, R@M, NDCG@M por consulta. M es el número de
# documentos relevantes encontrados en el archivo de juicios de relevancia por consulta. Luego calcule MAP
# como una métrica general.

# NOTA I: Para P@M y R@M suponga una escala de relevancia binaria. Los documentos que no se
# encuentran en el archivo “relevance-judgments” NO son relevantes para una consulta determinada.
# NOTA II: Para NDCG@M utilice la escala de relevancia no binaria que se encuentra en el archivo
# “relevance-judgments”.

# PROCESAMIENTO INICIAL DEL GROUND-TRUTH
ground_truth_df = pd.read_csv(ground_truth_path, sep='\t', header=None, names=['query_id', 'doc_id_relevance'])
ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
ground_truth_df['relevance'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
ground_truth_df[['query_id', 'doc_ids', 'relevance']]

,query_id,doc_ids,relevance
0,q01,"[d186, d254, d016]","[4, 5, 5]"
1,q02,"[d136, d139, d143, d283, d228, d164, d318, d29...","[2, 2, 4, 4, 4, 4, 2, 4, 4, 2, 2]"
2,q03,"[d152, d291, d283, d147, d318, d105]","[3, 4, 4, 3, 2, 2]"
3,q04,"[d275, d010, d286, d019, d049, d330, d270]","[3, 3, 2, 2, 2, 2, 3]"
4,q06,"[d069, d233, d257, d297, d026, d329]","[2, 3, 2, 3, 4, 5]"
5,q07,"[d004, d077, d266, d179]","[3, 3, 2, 3]"
6,q08,"[d205, d005, d110, d108, d117, d081, d292, d25...","[2, 4, 4, 3, 3, 2, 2, 5, 3, 3, 2, 2]"
7,q09,"[d205, d199, d198, d223, d217, d177]","[3, 5, 3, 2, 2, 2]"
8,q10,"[d068, d100, d065, d076, d231, d199, d052, d215]","[2, 2, 3, 3, 4, 4, 2, 2]"
9,q12,"[d239, d277, d258, d250]","[4, 4, 3, 4]"


# Calculo de la metrica de evaluación 𝑃@𝑀
Teniendo en memoria la información del "Ground-truth" mediante una represetnación de DataFrame, esto peritte a continuación contrastar la información procesada mediante la metrica de evalaución PRECISION, los resultados son mostrados en un DataFrame para facilidad de lectura.
Este código calcula la precisión 𝑃@𝑀 para cada consulta y la guarda en un DataFrame, comparando los documentos recuperados con los relevantes.

In [86]:
# Calculo de P@M

def limpiar_identificadores(doc_ids):
    return list(set([doc.replace(')', '').strip() for doc in doc_ids]))

def calcular_precision_p_m(resultados_df, ground_truth_df):
    precision_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)   
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids']   
        M = len(doc_retrieved)     
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]   
        if ground_truth_row.empty:
            precision_por_query[query_id] = 0.0
            continue
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved[:M]]
        precision_p_m = sum(relevance_list) / M if M > 0 else 0.0
        precision_por_query[query_id] = precision_p_m
            
    return precision_por_query

precision_resultados = calcular_precision_p_m(resultados_df, ground_truth_df)
precisionDF = pd.DataFrame(list(precision_resultados.items()), columns=['query_id', 'precision'])
precisionDF

,query_id,precision
0,q01,0.041667
1,q02,0.055838
2,q03,1.000000
3,q04,0.018519
4,q06,0.100000
5,q07,0.020408
6,q08,0.052023
7,q09,0.109091
8,q10,0.020513
9,q12,0.111111


# Calculo de la metrica de evaluación 𝑅@𝑀
De la misma manera, a continuación se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑅@𝑀 (Recall), los resultados son mostrados en un DataFrame para facilidad de lectura.




In [87]:
# Calculo de R@M

def calcular_recall_p_m(resultados_df, ground_truth_df):
    recall_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids'] 
        relevancia_docs = ground_truth_df[ground_truth_df['query_id'] == query_id]['doc_ids'].values[0]
        num_total_relevant_docs = len(relevancia_docs)
        if num_total_relevant_docs == 0:
            recall_por_query[query_id] = 0.0
            continue
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved]
        recall_p_m = sum(relevance_list) / num_total_relevant_docs
        recall_por_query[query_id] = recall_p_m
    
    return recall_por_query

recall_resultados = calcular_recall_p_m(resultados_df, ground_truth_df)
recallDF = pd.DataFrame(list(recall_resultados.items()), columns=['query_id', 'recall'])
recallDF


,query_id,recall
0,q01,0.666667
1,q02,1.000000
2,q03,1.000000
3,q04,0.571429
4,q06,0.666667
5,q07,0.250000
6,q08,0.750000
7,q09,1.000000
8,q10,0.500000
9,q12,1.000000


# Calculo de la metrica de evaluación 𝑁𝐷𝐶𝐺@𝑀
En la misma proporción, se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑁𝐷𝐶𝐺@𝑀 (Normalized Discounted Cumulative Gain at M), los resultados son mostrados en un DataFrame para facilidad de lectura. Este código calcula NDCG@M para cada consulta y lo almacena en un DataFrame. Utiliza la relevancia de los documentos recuperados y los compara con el orden ideal para cada consulta

In [88]:
# Calculo del NDCG@M

def calcular_dcg(relevancias, M):
    return sum((relevancia / np.log2(idx + 2)) for idx, relevancia in enumerate(relevancias[:M]))

def calcular_ndcg_p_m(resultados_df, ground_truth_df):
    ndcg_por_query = {}
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')]) 
    ground_truth_df['relevancias'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids'] 
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]
        if ground_truth_row.empty:
            ndcg_por_query[query_id] = 0.0
            continue       
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevancias_reales = ground_truth_row['relevancias'].values[0]

        M = len(relevancia_docs)      
        relevancias_obtenidas = [relevancias_reales[relevancia_docs.index(doc)] if doc in relevancia_docs else 0 for doc in doc_retrieved[:M]]
        dcg = calcular_dcg(relevancias_obtenidas, M)
        relevancias_ideales = sorted(relevancias_reales, reverse=True)
        idcg = calcular_dcg(relevancias_ideales, M)
        ndcg_p_m = dcg / idcg if idcg > 0 else 0.0
        ndcg_por_query[query_id] = ndcg_p_m
    
    return ndcg_por_query

ndcg_resultados = calcular_ndcg_p_m(resultados_df, ground_truth_df)
ndcgDF = pd.DataFrame(list(ndcg_resultados.items()), columns=['query_id', 'ndcg'])
ndcgDF

,query_id,ndcg
0,q01,0.000000
1,q02,0.000000
2,q03,0.852977
3,q04,0.212609
4,q06,0.182461
5,q07,0.118739
6,q08,0.098303
7,q09,0.000000
8,q10,0.000000
9,q12,0.000000


# Calculo de la metrica de evaluación 𝑀𝐴𝑃
Finalmente, se realiza el procesamiento de la información para aplicar la metrica de evaluación 𝑀𝐴𝑃 (Mean Average Precision), promediando las precisiones acumuladas de todas las consultas, devolviendo una métrica global de precisión, mostrando el resultado final general de evaluación del ejercicio.

In [89]:
# Calculo de MAP

def calcular_precision_acumulada(relevance_list):
    precisiones = []
    num_relevant = 0
    for i, relevancia in enumerate(relevance_list):
        if relevancia == 1:
            num_relevant += 1
            precisiones.append(num_relevant / (i + 1))  
    if precisiones:
        return sum(precisiones) / len(precisiones)
    else:
        return 0.0

def calcular_map(resultados_df, ground_truth_df):
    average_precisions = []
    if 'doc_ids' not in resultados_df.columns:
        resultados_df['doc_ids'] = resultados_df['docs'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')]) 
    resultados_df['doc_ids'] = resultados_df['doc_ids'].apply(limpiar_identificadores)
    if 'doc_ids' not in ground_truth_df.columns:
        ground_truth_df['doc_ids'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [doc.split(':')[0] for doc in x.split(',')])
    ground_truth_df['relevancias'] = ground_truth_df['doc_id_relevance'].apply(lambda x: [int(doc.split(':')[1]) for doc in x.split(',')])
    for _, row in resultados_df.iterrows():
        query_id = row['query_id']
        doc_retrieved = row['doc_ids']  
        ground_truth_row = ground_truth_df[ground_truth_df['query_id'] == query_id]
        if ground_truth_row.empty:
            average_precisions.append(0.0)
            continue
        relevancia_docs = ground_truth_row['doc_ids'].values[0]
        relevance_list = [1 if doc in relevancia_docs else 0 for doc in doc_retrieved]
        avg_precision = calcular_precision_acumulada(relevance_list)
        average_precisions.append(avg_precision)
    map_score = sum(average_precisions) / len(average_precisions) if average_precisions else 0.0
    return map_score

# Ejemplo de uso
map_score = calcular_map(resultados_df, ground_truth_df)
print("\nMAP (Mean Average Precision):", map_score)


MAP (Mean Average Precision): 0.14780139942128634
